In [86]:
from pinecone import Pinecone
from langchain.vectorstores import Pinecone as PineconeStore
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv
import os
from uuid import uuid4
import time

In [87]:
load_dotenv()

True

In [88]:
import pandas as pd
df = pd.read_csv('/Users/harshshivhare/SHL-AI/shl_enhanced_assessments.csv')

In [89]:
print(f"Original rows: {len(df)}")
duplicates = df.duplicated(subset=["Test Name", "Test Link"])
print(f"Duplicate rows: {duplicates.sum()}")

# Drop duplicates
df_clean = df.drop_duplicates(subset=["Test Name", "Test Link"])

# Optional: Also drop empty descriptions
df_clean = df_clean[df_clean["Description"].notna() & (df_clean["Description"].str.strip() != "")]

print(f"Cleaned rows: {len(df_clean)}")

Original rows: 794
Duplicate rows: 276
Cleaned rows: 518


In [90]:
df_clean.to_csv("shl_enhanced_assessments_clean.csv", index=False)
print("✅ Cleaned CSV saved as 'shl_enhanced_assessments_clean.csv'")

✅ Cleaned CSV saved as 'shl_enhanced_assessments_clean.csv'


In [91]:
from langchain.schema import Document

In [92]:
import pandas as pd
df = pd.read_csv('/Users/harshshivhare/SHL-AI/shl_final.csv')

In [113]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [31]:
from pinecone import Pinecone, ServerlessSpec


In [32]:
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

index_name = "shl"
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
        deletion_protection="enabled"
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [79]:
documents = []
ids = []

for _, row in df.iterrows():
    doc = Document(
        page_content=f"{row['Test Name']}. {row['Description']}",
        metadata = {
        "Test Name": str(row['Test Name']),
        "Test Link": str(row['Test Link']),
        "Description": str(row['Description']), 
        "Assessment Length": str(row['Assessment Length']),
        "Job Levels": str(row['Job Levels']),
        "Remote Testing": str(row['Remote Testing']),
        "Adaptive/IRT": str(row['Adaptive/IRT']),
        "Test Type": str(row['Test Type'])
    }
    )
    documents.append(doc)
    ids.append(str(uuid4()))

In [80]:
documents

[Document(metadata={'Test Name': 'Account Manager Solution', 'Test Link': 'https://www.shl.com/solutions/products/product-catalog/view/account-manager-solution/', 'Description': 'The Account Manager solution is an assessment used for job candidates applying to mid-level leadership positions that tend to manage the day-to-day operations and activities of client accounts. Sample tasks for these jobs include, but are not limited to: communicating with clients about project status, developing and maintaining project plans, coordinating internally with appropriate project personnel, and ensuring client expectations are being met. Potential job titles that use this solution are: Account Executive, Account Manager, and Senior Account Manager. There are multiple configurations of this solution available.', 'Assessment Length': 'Approximate Completion Time in minutes = 49', 'Job Levels': 'Mid-Professional,', 'Remote Testing': 'Yes', 'Adaptive/IRT': 'Yes', 'Test Type': 'CPAB'}, page_content='Acc

In [81]:
len(documents)

518

In [105]:
model = SentenceTransformer("all-MiniLM-L6-v2")

batch_size = 25
vectors_to_upsert = []

for i, row in df.iterrows():
    row = row.fillna("")
    text = f"{row['Test Name']}. {row['Description']}"
    embedding = model.encode(text).tolist()

    metadata = {
        "Test Name": str(row['Test Name']),
        "Test Link": str(row['Test Link']),
        "Description": str(row['Description']),
        "Assessment Length": str(row['Assessment Length']),
        "Job Levels": str(row['Job Levels']),
        "Remote Testing": str(row['Remote Testing']),
        "Adaptive/IRT": str(row['Adaptive/IRT']),
        "Test Type": str(row['Test Type'])
    }

    vector = (str(uuid4()), embedding, metadata)
    vectors_to_upsert.append(vector)

    if len(vectors_to_upsert) == batch_size or i == len(df) - 1:
        print(f"🔼 Upserting batch {i + 1 - batch_size + 1} to {i + 1}")
        try:
            index.upsert(vectors=vectors_to_upsert)
        except Exception as e:
            print(f"❌ Error during upsert: {e}")
            print("⏳ Retrying by refreshing Pinecone client...")
            try:
                pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
                index = pc.Index(index_name)
                index.upsert(vectors=vectors_to_upsert)
                print("✅ Retry succeeded.")
            except Exception as inner_e:
                print(f"❌ Retry failed again: {inner_e}")
                break
        vectors_to_upsert = []


🔼 Upserting batch 1 to 25
🔼 Upserting batch 26 to 50
❌ Error during upsert: Failed to connect; did you specify the correct index name?
⏳ Retrying by refreshing Pinecone client...
✅ Retry succeeded.
🔼 Upserting batch 51 to 75
🔼 Upserting batch 76 to 100
❌ Error during upsert: Failed to connect; did you specify the correct index name?
⏳ Retrying by refreshing Pinecone client...
✅ Retry succeeded.
🔼 Upserting batch 101 to 125
🔼 Upserting batch 126 to 150
🔼 Upserting batch 151 to 175
🔼 Upserting batch 176 to 200
🔼 Upserting batch 201 to 225
🔼 Upserting batch 226 to 250
🔼 Upserting batch 251 to 275
❌ Error during upsert: Failed to connect; did you specify the correct index name?
⏳ Retrying by refreshing Pinecone client...
✅ Retry succeeded.
🔼 Upserting batch 276 to 300
🔼 Upserting batch 301 to 325
🔼 Upserting batch 326 to 350
🔼 Upserting batch 351 to 375
🔼 Upserting batch 376 to 400
🔼 Upserting batch 401 to 425
🔼 Upserting batch 426 to 450
🔼 Upserting batch 451 to 475
🔼 Upserting batch 476 

## ReRanking using Pincone Library

In [108]:
query="Looking to hire mid-level professionals who are proficient in Python, SQL and Java Script. Need an assessment package that can test all skills with max duration of 60 minutes."
embedder = SentenceTransformer("all-MiniLM-L6-v2")
query_vector = embedder.encode(query).tolist()

# Step 1: Retrieve Top-K Candidates
retrieved = index.query(
    vector=query_vector,
    top_k=10,
    include_metadata=True
)

# Step 2: Prepare Documents for Reranking
transformed_documents = []
for i, match in enumerate(retrieved['matches']):
    metadata = match['metadata']
    transformed_documents.append({
        "id": str(i),
        "reranking_field": f"{metadata['Test Name']}. {metadata['Description']}",
        "metadata": metadata
    })

# Step 3: Call Pinecone Reranker
reranked_results_field = pc.inference.rerank(
    model="bge-reranker-v2-m3",
    query=query,
    documents=transformed_documents,
    rank_fields=["reranking_field"],
    top_n=5,
    return_documents=True
)

# Step 4: Show Reranked Results
def show_reranked_results(question, matches):
    print(f'\n🧠 Query: \'{question}\'')
    print('\n🔢 Reranked Results:\n')
    for i, match in enumerate(matches):
        doc = match.document
        metadata = doc.metadata
        print(f'{str(i+1).rjust(3)}. {metadata.get("Test Name")}')
        print(f'     🔢 Score: {match.score:.4f}')
        print(f'     🕒 Length: {metadata.get("Assessment Length")} | 🧑‍💼 Level: {metadata.get("Job Levels")}')
        print(f'     🔗 {metadata.get("Test Link")}\n')

show_reranked_results(query, reranked_results_field.data)


🧠 Query: 'Looking to hire mid-level professionals who are proficient in Python, SQL and Java Script. Need an assessment package that can test all skills with max duration of 60 minutes.'

🔢 Reranked Results:

  1. Python (New)
     🔢 Score: 0.0595
     🕒 Length: Approximate Completion Time in minutes = 11 | 🧑‍💼 Level: Mid-Professional, Professional Individual Contributor,
     🔗 https://www.shl.com/solutions/products/product-catalog/view/python-new/

  2. Technology Professional 8.0 Job Focused Assessment
     🔢 Score: 0.0148
     🕒 Length: Approximate Completion Time in minutes = 16 | 🧑‍💼 Level: Entry-Level, Graduate, Mid-Professional, Professional Individual Contributor,
     🔗 https://www.shl.com/solutions/products/product-catalog/view/technology-professional-8-0-job-focused-assessment/

  3. Professional + 7.0 Solution
     🔢 Score: 0.0088
     🕒 Length: Approximate Completion Time in minutes = 51 | 🧑‍💼 Level: Mid-Professional, Professional Individual Contributor,
     🔗 https://w

## Indexing Retrieval using BM25

In [120]:
import numpy as np
from typing import List
from langchain_core.documents import Document
from rank_bm25 import BM25Okapi

class FusionRetriever:
    def __init__(self, pinecone_index, sentence_model, documents: List[Document], alpha: float = 0.5):
        """
        Args:
        - pinecone_index: a Pinecone Index() object (not LangChain vectorstore)
        - sentence_model: SentenceTransformer embedding model
        - documents: List of Documents used for indexing
        - alpha: fusion weighting factor between vector and BM25 scores
        """
        self.index = pinecone_index
        self.model = sentence_model
        self.documents = documents
        self.alpha = alpha
        self.bm25 = self._create_bm25_index(documents)

    def _create_bm25_index(self, documents: List[Document]) -> BM25Okapi:
        tokenized_docs = [doc.page_content.split() for doc in documents if doc.page_content]
        return BM25Okapi(tokenized_docs)

    def retrieve(self, query: str, k: int = 5, fetch_k: int = 30) -> List[Document]:
        epsilon = 1e-8

        # Step 1: Encode query using the same SentenceTransformer used during indexing
        query_vector = self.model.encode(query).tolist()

        # Step 2: Query Pinecone directly
        response = self.index.query(
            vector=query_vector,
            top_k=fetch_k,
            include_metadata=True
        )

        if not response['matches']:
            print("⚠️ No results from vector store.")
            return []

        all_docs = []
        vector_scores = []

        for match in response['matches']:
            metadata = match['metadata']
            content = f"{metadata.get('Test Name', '')}. {metadata.get('Description', '')}"
            doc = Document(page_content=content, metadata=metadata)
            all_docs.append(doc)
            vector_scores.append(match['score'])

        # Normalize vector scores (inverted so higher = better)
        vector_scores = np.array(vector_scores)
        vector_scores = 1 - (vector_scores - np.min(vector_scores)) / (np.max(vector_scores) - np.min(vector_scores) + epsilon)

        # Step 3: BM25 scoring
        query_tokens = query.split()
        mini_bm25 = BM25Okapi([doc.page_content.split() for doc in all_docs])
        bm25_scores = mini_bm25.get_scores(query_tokens)

        bm25_scores = np.array(bm25_scores)
        if np.max(bm25_scores) > 0:
            bm25_scores = (bm25_scores - np.min(bm25_scores)) / (np.max(bm25_scores) - np.min(bm25_scores) + epsilon)
        else:
            bm25_scores = np.zeros_like(vector_scores)

        # Step 4: Combine scores
        combined_scores = self.alpha * vector_scores + (1 - self.alpha) * bm25_scores
        sorted_indices = np.argsort(combined_scores)[::-1]

        return [all_docs[i] for i in sorted_indices[:k]]


In [114]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = PineconeVectorStore(index=index, embedding=embedding)


In [122]:
model = SentenceTransformer("all-MiniLM-L6-v2")

# Run retrieval
retriever = FusionRetriever(index, model, documents, alpha=0.5)
results = retriever.retrieve("Looking to hire mid-level professionals who are proficient in Python, SQL and Java Script. Need an assessment package that can test all skills with max duration of 60 minutes.", k=5)

# Display
for i, doc in enumerate(results):
    print(f"\n🔹 {i+1}. {doc.metadata['Test Name']}")
    print(f"   🕒 {doc.metadata['Assessment Length']}")
    print(f"   🧑‍💼 {doc.metadata['Job Levels']}")
    print(f"   🔗 {doc.metadata['Test Link']}")


🔹 1. PJM Selection Report
   🕒 
   🧑‍💼 Mid-Professional, Manager,
   🔗 https://www.shl.com/solutions/products/product-catalog/view/pjm-selection-report/

🔹 2. Global Skills Development Report
   🕒 
   🧑‍💼 Director, Entry-Level, Executive, General Population, Graduate, Manager, Mid-Professional, Front Line Manager, Supervisor,
   🔗 https://www.shl.com/solutions/products/product-catalog/view/global-skills-development-report/

🔹 3. Verify - Following Instructions
   🕒 Approximate Completion Time in minutes = 8
   🧑‍💼 Entry-Level,
   🔗 https://www.shl.com/solutions/products/product-catalog/view/verify-following-instructions/

🔹 4. RemoteWorkQ
   🕒 Approximate Completion Time in minutes = 10
   🧑‍💼 Manager, Mid-Professional, Professional Individual Contributor, Supervisor, Front Line Manager, General Population, Graduate, Director, Entry-Level, Executive,
   🔗 https://www.shl.com/solutions/products/product-catalog/view/remoteworkq/

🔹 5. Verify - Numerical Ability
   🕒 Approximate Completi

In [123]:
retriever

In [132]:
import pandas as pd
import json
import os
import re
import google.generativeai as genai
from dotenv import load_dotenv
from tqdm import tqdm

# === Load env + CSV ===
load_dotenv()
df = pd.read_csv("shl_enhanced_assessments_clean.csv")

# === Gemini API Setup ===
api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)
llm = genai.GenerativeModel(model_name="gemini-2.0-flash", generation_config={"temperature": 0.9})

# === Prompt template ===
def generate_prompt(title: str, description: str) -> str:
    return f"""
You are an intelligent assistant designed to improve AI-based search and recommendation of assessments.

Your task is to analyze the given assessment title and description, and extract a concise list of relevant tags. These tags will help match assessments to user queries like:
“I am hiring for Java developers who can also collaborate effectively with my business teams. Looking for an assessment(s) that can be completed in 40 minutes.”

Extract tags that accurately reflect:

Hard skills (e.g., Java, SQL, REST APIs)

Soft skills (e.g., collaboration, communication)

Job roles or levels (e.g., backend developer, mid-level, team lead)

Business domain or context (e.g., cross-functional, agile)

Assessment traits (e.g., time limit, difficulty level, scenario-based)

Be specific and comprehensive. Avoid generic or vague tags like “test” or “assessment.” Use domain-relevant terminology.

Return output ONLY in this JSON format:
{{"tags": ["tag1", "tag2", "tag3"]}}

Title: "{title}"
Description: "{description}"
"""

tag_column = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    title = str(row.get("Test Name", ""))
    description = str(row.get("Description", ""))

    prompt = generate_prompt(title, description)

    try:
        response = llm.generate_content(prompt)
        response_text = response.text.strip()

        # Remove code block markers
        response_text = re.sub(r"^```json", "", response_text)
        response_text = re.sub(r"```$", "", response_text)
        response_text = response_text.strip()

        parsed = json.loads(response_text)
        tags = parsed.get("tags", [])
    except Exception as e:
        tags = []
        print(f"Error on '{title}': {e}")

    tag_column.append(tags)

100%|██████████| 518/518 [08:36<00:00,  1.00it/s]


In [133]:
tag_column

[['Account Management',
  'Client Communication',
  'Project Management',
  'Coordination',
  'Client Expectations',
  'Leadership',
  'Mid-Level',
  'Account Executive',
  'Account Manager',
  'Senior Account Manager'],
 ['Administrative Assistant',
  'Secretary',
  'Office Manager',
  'Administrative Aide',
  'Administrative Associate',
  'Entry-Level',
  'Mid-Level',
  'Clerical Skills',
  'Administrative Skills',
  'Office Management',
  'Customer Service',
  'Communication Skills',
  'Scheduling',
  'Meeting Arrangement',
  'Correspondence',
  'Office Coordination',
  'Greeting Visitors'],
 ['sales management',
  'mid-level',
  'front line management',
  'financial activities',
  'branch management',
  'brokerage management',
  'risk management',
  'insurance management',
  'credit department',
  'agency manager',
  'brokerage manager'],
 ['entry-level',
  'apprenticeship',
  'cognitive ability',
  'global',
  'multi-national',
  'job-focused',
  'short assessment'],
 ['entry-leve

In [134]:
df["Tags"] = tag_column
df.to_csv("shl_enhanced_assessments_with_tags.csv", index=False)
print("\n✅ Enrichment complete. Tags saved to 'shl_enhanced_assessments_with_tags.csv'")


✅ Enrichment complete. Tags saved to 'shl_enhanced_assessments_with_tags.csv'


In [135]:
import pandas as pd
from langchain_core.documents import Document
from uuid import uuid4
import ast  # in case tags are stored as strings like "['python', 'sql']"

# Load enriched CSV
df = pd.read_csv('/Users/harshshivhare/SHL-AI/shl_enhanced_assessments_with_tags.csv')

documents = []
ids = []

for _, row in df.iterrows():
    # Safely parse the Tags column
    try:
        tags = ast.literal_eval(row["Tags"]) if isinstance(row["Tags"], str) else row["Tags"]
        if not isinstance(tags, list):
            tags = []
    except Exception:
        tags = []

    # Create document with all metadata including tags
    doc = Document(
        page_content=f"{row['Test Name']}. {row['Description']}",
        metadata={
            "Test Name": str(row['Test Name']),
            "Test Link": str(row['Test Link']),
            "Description": str(row['Description']), 
            "Assessment Length": str(row['Assessment Length']),
            "Job Levels": str(row['Job Levels']),
            "Remote Testing": str(row['Remote Testing']),
            "Adaptive/IRT": str(row['Adaptive/IRT']),
            "Test Type": str(row['Test Type']),
            "Tags": tags  # ✅ Include tags for retrieval & reranking
        }
    )
    documents.append(doc)
    ids.append(str(uuid4()))


In [165]:
documents

[Document(metadata={'Test Name': 'Account Manager Solution', 'Test Link': 'https://www.shl.com/solutions/products/product-catalog/view/account-manager-solution/', 'Description': 'The Account Manager solution is an assessment used for job candidates applying to mid-level leadership positions that tend to manage the day-to-day operations and activities of client accounts. Sample tasks for these jobs include, but are not limited to: communicating with clients about project status, developing and maintaining project plans, coordinating internally with appropriate project personnel, and ensuring client expectations are being met. Potential job titles that use this solution are: Account Executive, Account Manager, and Senior Account Manager. There are multiple configurations of this solution available.', 'Assessment Length': 'Approximate Completion Time in minutes = 49', 'Job Levels': 'Mid-Professional,', 'Remote Testing': 'Yes', 'Adaptive/IRT': 'Yes', 'Test Type': 'CPAB', 'Tags': ['Account 

In [167]:
from sentence_transformers import SentenceTransformer
from uuid import uuid4
import ast

model = SentenceTransformer("all-MiniLM-L6-v2")
batch_size = 25
vectors_to_upsert = []

for i, row in df.iterrows():
    row = row.fillna("")
    text = f"{row['Test Name']}. {row['Description']}"
    embedding = model.encode(text).tolist()

    # ✅ Parse tags from CSV
    tags_raw = row.get("Tags", "")
    try:
        tags = ast.literal_eval(tags_raw) if isinstance(tags_raw, str) else tags_raw
        if not isinstance(tags, list):
            tags = []
    except Exception:
        tags = []

    metadata = {
        "Test Name": str(row['Test Name']),
        "Test Link": str(row['Test Link']),
        "Description": str(row['Description']),
        "Assessment Length": str(row['Assessment Length']),
        "Job Levels": str(row['Job Levels']),
        "Remote Testing": str(row['Remote Testing']),
        "Adaptive/IRT": str(row['Adaptive/IRT']),
        "Test Type": str(row['Test Type']),
        "Tags": tags  # ✅ Correct, per row
    }

    vector = (str(uuid4()), embedding, metadata)
    vectors_to_upsert.append(vector)

    # ⏫ Batched upsert
    if len(vectors_to_upsert) == batch_size or i == len(df) - 1:
        print(f"🔼 Upserting batch {i + 1 - batch_size + 1} to {i + 1}")
        try:
            index.upsert(vectors=vectors_to_upsert)
        except Exception as e:
            print(f"❌ Error during upsert: {e}")
            print("⏳ Retrying by refreshing Pinecone client...")
            try:
                pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
                index = pc.Index(index_name)
                index.upsert(vectors=vectors_to_upsert)
                print("✅ Retry succeeded.")
            except Exception as inner_e:
                print(f"❌ Retry failed again: {inner_e}")
                break
        vectors_to_upsert = []


🔼 Upserting batch 1 to 25
🔼 Upserting batch 26 to 50
🔼 Upserting batch 51 to 75
🔼 Upserting batch 76 to 100
❌ Error during upsert: Failed to connect; did you specify the correct index name?
⏳ Retrying by refreshing Pinecone client...
✅ Retry succeeded.
🔼 Upserting batch 101 to 125
🔼 Upserting batch 126 to 150
🔼 Upserting batch 151 to 175
🔼 Upserting batch 176 to 200
🔼 Upserting batch 201 to 225
🔼 Upserting batch 226 to 250
❌ Error during upsert: Failed to connect; did you specify the correct index name?
⏳ Retrying by refreshing Pinecone client...
✅ Retry succeeded.
🔼 Upserting batch 251 to 275
🔼 Upserting batch 276 to 300
🔼 Upserting batch 301 to 325
🔼 Upserting batch 326 to 350
🔼 Upserting batch 351 to 375
❌ Error during upsert: Failed to connect; did you specify the correct index name?
⏳ Retrying by refreshing Pinecone client...
✅ Retry succeeded.
🔼 Upserting batch 376 to 400
🔼 Upserting batch 401 to 425
🔼 Upserting batch 426 to 450
🔼 Upserting batch 451 to 475
🔼 Upserting batch 476 

## ReRanking using Pincone Library

In [168]:
query="Looking to hire mid-level professionals who are proficient in Python, SQL and Java Script. Need an assessment package that can test all skills with max duration of 60 minutes."
embedder = SentenceTransformer("all-MiniLM-L6-v2")
query_vector = embedder.encode(query).tolist()

# Step 1: Retrieve Top-K Candidates
retrieved = index.query(
    vector=query_vector,
    top_k=10,
    include_metadata=True
)

# Step 2: Prepare Documents for Reranking
transformed_documents = []
for i, match in enumerate(retrieved['matches']):
    metadata = match['metadata']
    tags = metadata.get("Tags", [])
    tag_string = " ".join(tags) if isinstance(tags, list) else str(tags)
    
    transformed_documents.append({
        "id": str(i),
        "reranking_field": f"{metadata['Test Name']}. {metadata['Description']}. Tags: {tag_string}",
        "metadata": metadata
    })


# Step 3: Call Pinecone Reranker
reranked_results_field = pc.inference.rerank(
    model="bge-reranker-v2-m3",
    query=query,
    documents=transformed_documents,
    rank_fields=["reranking_field"],
    top_n=5,
    return_documents=True
)

# Step 4: Show Reranked Results
def show_reranked_results(question, matches):
    print(f'\n🧠 Query: \'{question}\'')
    print('\n🔢 Reranked Results:\n')
    for i, match in enumerate(matches):
        doc = match.document
        metadata = doc.metadata
        print(f'{str(i+1).rjust(3)}. {metadata.get("Test Name")}')
        print(f'     🔢 Score: {match.score:.4f}')
        print(f'     🕒 Length: {metadata.get("Assessment Length")} | 🧑‍💼 Level: {metadata.get("Job Levels")}')
        print(f'     🔗 {metadata.get("Test Link")}\n')

show_reranked_results(query, reranked_results_field.data)


🧠 Query: 'Looking to hire mid-level professionals who are proficient in Python, SQL and Java Script. Need an assessment package that can test all skills with max duration of 60 minutes.'

🔢 Reranked Results:

  1. Verify - Deductive Reasoning
     🔢 Score: 0.0256
     🕒 Length: Approximate Completion Time in minutes = 20 | 🧑‍💼 Level: Director, Entry-Level, Executive, Front Line Manager, General Population, Graduate, Manager, Mid-Professional, Professional Individual Contributor, Supervisor,
     🔗 https://www.shl.com/solutions/products/product-catalog/view/verify-deductive-reasoning/

  2. Python (New)
     🔢 Score: 0.0228
     🕒 Length: Approximate Completion Time in minutes = 11 | 🧑‍💼 Level: Mid-Professional, Professional Individual Contributor,
     🔗 https://www.shl.com/solutions/products/product-catalog/view/python-new/

  3. Technology Professional 8.0 Job Focused Assessment
     🔢 Score: 0.0056
     🕒 Length: Approximate Completion Time in minutes = 16 | 🧑‍💼 Level: Entry-Level,

## Hybrid Retrieval(Indexing Retrieval using BM25)

In [169]:
import numpy as np
from typing import List
from langchain_core.documents import Document
from rank_bm25 import BM25Okapi

class FusionRetriever:
    def __init__(self, pinecone_index, sentence_model, documents: List[Document], alpha: float = 0.5):
        """
        Args:
        - pinecone_index: a Pinecone Index() object (not LangChain vectorstore)
        - sentence_model: SentenceTransformer embedding model
        - documents: List of Documents used for indexing
        - alpha: fusion weighting factor between vector and BM25 scores
        """
        self.index = pinecone_index
        self.model = sentence_model
        self.documents = documents
        self.alpha = alpha
        self.bm25 = self._create_bm25_index(documents)

    def _create_bm25_index(self, documents: List[Document]) -> BM25Okapi:
        tokenized_docs = []
        for doc in documents:
            base_tokens = doc.page_content.split() if doc.page_content else []
            tags = doc.metadata.get("Tags", [])
            tag_tokens = tags if isinstance(tags, list) else []
            combined_tokens = base_tokens + tag_tokens * 3  # Prefer tags
            tokenized_docs.append(combined_tokens)
        return BM25Okapi(tokenized_docs)

    def retrieve(self, query: str, k: int = 5, fetch_k: int = 30) -> List[Document]:
        epsilon = 1e-8

        # Step 1: Encode query using the same SentenceTransformer used during indexing
        query_vector = self.model.encode(query).tolist()

        # Step 2: Query Pinecone directly
        response = self.index.query(
            vector=query_vector,
            top_k=fetch_k,
            include_metadata=True
        )

        if not response['matches']:
            print("⚠️ No results from vector store.")
            return []

        all_docs = []
        vector_scores = []

        for match in response['matches']:
            metadata = match['metadata']
            content = f"{metadata.get('Test Name', '')}. {metadata.get('Description', '')}"
            doc = Document(page_content=content, metadata=metadata)
            all_docs.append(doc)
            vector_scores.append(match['score'])

        # Normalize vector scores (inverted so higher = better)
        vector_scores = np.array(vector_scores)
        vector_scores = 1 - (vector_scores - np.min(vector_scores)) / (np.max(vector_scores) - np.min(vector_scores) + epsilon)

        # Step 3: BM25 scoring
        query_tokens = query.split()
        mini_bm25 = BM25Okapi([doc.page_content.split() for doc in all_docs])
        bm25_scores = mini_bm25.get_scores(query_tokens)

        bm25_scores = np.array(bm25_scores)
        if np.max(bm25_scores) > 0:
            bm25_scores = (bm25_scores - np.min(bm25_scores)) / (np.max(bm25_scores) - np.min(bm25_scores) + epsilon)
        else:
            bm25_scores = np.zeros_like(vector_scores)

        # Step 4: Combine scores
        combined_scores = self.alpha * vector_scores + (1 - self.alpha) * bm25_scores
        sorted_indices = np.argsort(combined_scores)[::-1]

        return [all_docs[i] for i in sorted_indices[:k]]


In [170]:
model = SentenceTransformer("all-MiniLM-L6-v2")

# Run retrieval
retriever = FusionRetriever(index, model, documents, alpha=0.5)
results = retriever.retrieve("I am hiring for Java developers who can also collaborate effectively with my business teams. Looking for an assessment(s) that can be completed in 40 minutes.", k=5)

# Display
for i, doc in enumerate(results):
    print(f"\n🔹 {i+1}. {doc.metadata['Test Name']}")
    print(f"   🕒 {doc.metadata['Assessment Length']}")
    print(f"   🧑‍💼 {doc.metadata['Job Levels']}")
    print(f"   🔗 {doc.metadata['Test Link']}")


🔹 1. Apprentice 8.0 Job Focused Assessment
   🕒 Approximate Completion Time in minutes = 20
   🧑‍💼 Entry-Level, General Population, Graduate,
   🔗 https://www.shl.com/solutions/products/product-catalog/view/apprentice-8-0-job-focused-assessment/

🔹 2. Apprentice + 8.0 Job Focused Assessment
   🕒 Approximate Completion Time in minutes = 30
   🧑‍💼 General Population, Graduate, Entry-Level,
   🔗 https://www.shl.com/solutions/products/product-catalog/view/apprentice-8-0-job-focused-assessment-4261/

🔹 3. PJM Selection Report
   🕒 
   🧑‍💼 Mid-Professional, Manager,
   🔗 https://www.shl.com/solutions/products/product-catalog/view/pjm-selection-report/

🔹 4. MFS 360 UCF Performance Potential Dev Tips Report
   🕒 
   🧑‍💼 
   🔗 https://www.shl.com/solutions/products/product-catalog/view/mfs-360-ucf-performance-potential-dev-tips-report/

🔹 5. 360° Multi-Rater Feedback System (MFS)
   🕒 Approximate Completion Time in minutes = Untimed
   🧑‍💼 Director, Executive, Front Line Manager, Manager, Mid

## Enhancing Reranking

In [148]:
from typing import List

In [192]:
query='''I am hiring for Java developers who can also collaborate effectively with my business teams. Looking for an assessment(s) that can be completed in 40 minutes.
'''
model = SentenceTransformer("all-MiniLM-L6-v2")
query_vector = model.encode(query).tolist()

# === Step 1: Retrieve Top 30 from Pinecone ===
retrieved = index.query(
    vector=query_vector,
    top_k=30,
    include_metadata=True
)

# === Step 2: Format results for reranking ===
assessment_blocks = []
id_map = {}

for i, match in enumerate(retrieved['matches']):
    md = match['metadata']
    aid = str(i + 1)
    id_map[aid] = md
    tags = ", ".join(md.get("Tags", [])) if isinstance(md.get("Tags"), list) else md.get("Tags", "")
    block = f"""{aid}. Title: {md.get('Test Name', '')}
Description: {md.get('Description', '')}
Tags: {tags}
Job Level: {md.get('Job Levels', '')}
Duration: {md.get('Assessment Length', '')}
"""
    assessment_blocks.append(block)

# === Step 3: Create reranking prompt ===
def build_prompt(query, blocks):
    return f"""
You are an expert assistant helping HR teams and recruiters select the most relevant assessments for their hiring needs.

Given a natural language hiring query or Job description of a job and a list of available assessments, your task is to intelligently rank and recommend the top 10 most relevant assessments.

Assessments are described using their title, description, tags, target job level, and duration. The user query may include specific technical and soft skills, job roles, team collaboration needs, or constraints like duration (e.g., "within 40 minutes").

You must:

Match skills (technical and soft)

Understand job role and seniority

Respect duration constraints

Recognize contextual needs (e.g., cross-functional, remote-friendly, leadership focus)

Rank results based on semantic relevance, not just keyword overlap

Your job is to return the **top 10 most relevant assessments**, ranked by relevance to the query, in JSON format like this:

[
  {{"id": "3", "reason": "Matches Python, SQL and is under 60 mins"}},
  ...
]

Query:
"{query}"

Assessments:
{''.join(blocks)}
"""

prompt = build_prompt(query, assessment_blocks)

# === Step 4: Rerank using Gemini ===
response = llm.generate_content(prompt)
response_text = response.text.strip()

# Extract and clean JSON from Gemini response
response_text = re.sub(r"^```json", "", response_text)
response_text = re.sub(r"```$", "", response_text)
response_text = response_text.strip()

try:
    reranked = json.loads(response_text)
except Exception as e:
    print(f"❌ Failed to parse Gemini response: {e}")
    reranked = []

# === Step 5: Show Results ===
print(f"\n🧠 Query: {query}\n")
print("📊 Top Gemini-Reranked Assessments:\n")

for item in reranked:
    aid = item["id"]
    reason = item.get("reason", "No reason given")
    md = id_map.get(aid)

    if not md:
        continue

    print(f"🔹 {md.get('Test Name')}")
    print(f"   🧠 Reason: {reason}")
    print(f"   🕒 Length: {md.get('Assessment Length')}")
    print(f"   🧑‍💼 Level: {md.get('Job Levels')}")
    print(f"   🏷️ Tags: {', '.join(md.get('Tags', [])) if isinstance(md.get('Tags'), list) else md.get('Tags')}")
    print(f"   🔗 {md.get('Test Link')}\n")


🧠 Query: I am hiring for Java developers who can also collaborate effectively with my business teams. Looking for an assessment(s) that can be completed in 40 minutes.


📊 Top Gemini-Reranked Assessments:

🔹 Java 8 (New)
   🧠 Reason: Directly assesses Java 8 knowledge and fits the time constraint.
   🕒 Length: Approximate Completion Time in minutes = 18
   🧑‍💼 Level: Mid-Professional, Professional Individual Contributor,
   🏷️ Tags: Java 8, Java, Class Design, Exceptions, Generics, Collections, Concurrency, JDBC, Java I/O, Multi-choice, Fundamentals, Backend Developer
   🔗 https://www.shl.com/solutions/products/product-catalog/view/java-8-new/

🔹 Java 2 Platform Enterprise Edition 1.4 Fundamental
   🧠 Reason: Assesses Java EE 1.4 fundamentals, relevant for Java developers and under 40 minutes.
   🕒 Length: Approximate Completion Time in minutes = 30
   🧑‍💼 Level: Entry-Level, Mid-Professional, Professional Individual Contributor,
   🏷️ Tags: Java 2 Platform Enterprise Edition 1.4, J2E

In [161]:
assessment_blocks

['1. Title: Verify - General Ability Screen\nDescription: The General Ability Screen is a first for us – a measure of general mental ability or ‘g’ .\r\nTargeted at ‘entry-level’ roles, General Ability Screen is intended to precede other measures to provide process efficiency, and to ensure candidates experience the most positive possible assessment process, by being quick, fair and available as an online test 24/7/365. \r\nAs the name suggests, the test is designed to be used in screening or sifting processes\nTags: 360-degree feedback, multi-rater feedback, performance management, employee development, competency assessment, SHL Universal Competency Framework (UCF), manager feedback, peer feedback, direct report feedback, self-assessment, leadership assessment, communication skills, interpersonal skills, impact on others, strengths and weaknesses, development opportunities, all levels, all industries, holistic view, research-driven competencies, developmental feedback\nJob Level: Ent

In [157]:
def recall_at_k(predicted_ids, relevant_ids, k):
    top_k = predicted_ids[:k]
    return len(set(top_k) & set(relevant_ids)) / len(relevant_ids)

def average_precision_at_k(predicted_ids, relevant_ids, k):
    hits = 0
    sum_precisions = 0
    for i, pid in enumerate(predicted_ids[:k]):
        if pid in relevant_ids:
            hits += 1
            sum_precisions += hits / (i + 1)
    return sum_precisions / min(k, len(relevant_ids))

# Loop over all test queries
mean_recall = np.mean([recall_at_k(pred[i], true[i], k=10) for i in range(len(queries))])
mean_ap = np.mean([average_precision_at_k(pred[i], true[i], k=10) for i in range(len(queries))])


NameError: name 'queries' is not defined

In [162]:
df["Tags"].value_counts().head(5)


Tags
['Account Management', 'Client Communication', 'Project Management', 'Coordination', 'Client Expectations', 'Leadership', 'Mid-Level', 'Account Executive', 'Account Manager', 'Senior Account Manager']                                                                                                                                                                            1
['Manufacturing', 'Industrial', 'Safety', 'Domain Expertise', 'Practical Solutions', 'Multitasking', 'Machining and Equipment Operators', 'Laborer/Warehouse', 'Assemblers and Fitters', 'Maintenance/Repair Workers', 'Dispatchers', 'Surveillance', 'Quality Assurance Workers', 'Material Handlers', 'Truck/Ship Loaders', 'Job-Focused Assessment', 'Behavioral Assessment']    1
['motivation', 'employee motivation', 'candidate assessment', 'feedback report', 'individual assessment', 'work motivators', 'work demotivators', 'personality assessment', 'behavioral assessment', 'employee engagement']                            

In [164]:
assessment_blocks = []
id_map = {}

for i, match in enumerate(retrieved['matches']):
    md = match['metadata']
    aid = str(i + 1)
    id_map[aid] = md
    tags = ", ".join(md.get("Tags", [])) if isinstance(md.get("Tags"), list) else md.get("Tags", "")
    # print("tags",tags)
    block = f"""{aid}. Title: {md.get('Test Name', '')}
    
Description: {md.get('Description', '')}
Tags: {tags}
Job Level: {md.get('Job Levels', '')}
Duration: {md.get('Assessment Length', '')}
"""
    print("block",block)
    assessment_blocks.append(block)

block 1. Title: Verify - General Ability Screen

Description: The General Ability Screen is a first for us – a measure of general mental ability or ‘g’ .
Targeted at ‘entry-level’ roles, General Ability Screen is intended to precede other measures to provide process efficiency, and to ensure candidates experience the most positive possible assessment process, by being quick, fair and available as an online test 24/7/365. 
As the name suggests, the test is designed to be used in screening or sifting processes
Tags: 360-degree feedback, multi-rater feedback, performance management, employee development, competency assessment, SHL Universal Competency Framework (UCF), manager feedback, peer feedback, direct report feedback, self-assessment, leadership assessment, communication skills, interpersonal skills, impact on others, strengths and weaknesses, development opportunities, all levels, all industries, holistic view, research-driven competencies, developmental feedback
Job Level: Entry-L

In [188]:
import re
import requests
from bs4 import BeautifulSoup
import google.generativeai as genai

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
llm = genai.GenerativeModel("gemini-2.0-flash")

def is_url(text: str) -> bool:
    return text.startswith("http://") or text.startswith("https://")

def is_probable_jd(text: str) -> bool:
    # Heuristic: if the text is long, contains bullet points or JD terms, treat as JD
    return (
        len(text.split()) > 50 or (
            "responsibilities" in text.lower() or
            "qualifications" in text.lower() or
            "job description" in text.lower() or
            "apply now" in text.lower() or
            "skills required" in text.lower()
        )
    )

def extract_jd_from_url(url: str) -> str:
    try:
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.content, "html.parser")

        # Heuristically try to extract large content blocks
        candidates = soup.find_all(["p", "div", "section", "article"], recursive=True)
        jd_candidates = [c.get_text(strip=True, separator=" ") for c in candidates if len(c.get_text(strip=True)) > 100]
        jd_text = "\n".join(jd_candidates[:5])  # Limit to top 5 blocks
        return jd_text
    except Exception as e:
        print(f"❌ Error fetching URL content: {e}")
        return ""

def llm_extract_query_from_jd(jd_text: str) -> str:
    prompt = f"""
You are an intelligent assistant that converts job descriptions into smart search queries to find suitable assessment for this JD.

Your job is to read the JD(Job Description) below and write a concise query that captures:
- The role or domain
- Hard skills
- Soft skills or traits (like communication, collaboration)
- Any constraints like seniority level or duration
- Combine these into one smart search query

It should be sentence-like, not a list. Use natural language.
For Example:"I am hiring for Java developers who can also collaborate effectively with my business teams. Looking 
for an assessment(s) that can be completed in 40 minutes."

Return ONLY the final query string. No commentary or explanations.


Job Description:
{jd_text}
"""
    try:
        print("⏳ Calling Gemini...")
        response = llm.generate_content(prompt)
        # print("response",response)
        response_text = response.text.strip()
        print("✅ Gemini returned:\n", response_text)

        return response.text.strip()
    except Exception as e:
        print(f"❌ LLM error: {e}")
        return ""

def preprocess_input(user_input: str) -> str:
    if is_url(user_input):
        print("🌐 Detected URL input — scraping JD...")
        jd_text = extract_jd_from_url(user_input)
        if not jd_text:
            return "Could not extract job description from URL."
        return llm_extract_query_from_jd(jd_text)

    elif is_probable_jd(user_input):
        print("📄 Detected JD text — parsing with LLM...")
        return llm_extract_query_from_jd(user_input)

    else:
        print("💬 Detected simple query — using as-is.")
        return user_input.strip()


In [190]:
query = preprocess_input("https://www.linkedin.com/jobs/view/research-engineer-ai-at-shl-4194768899/?originalSubdomain=in")
print("🧠 Final query:", query)

🌐 Detected URL input — scraping JD...
⏳ Calling Gemini...
✅ Gemini returned:
 I'm looking for an assessment for a mid-senior level Research Engineer with experience in AI/ML, specifically NLP, computer vision, and speech processing, who is proficient in Python and ML frameworks like TensorFlow, PyTorch, and OpenAI APIs, and possesses agile and proactive thinking.
🧠 Final query: I'm looking for an assessment for a mid-senior level Research Engineer with experience in AI/ML, specifically NLP, computer vision, and speech processing, who is proficient in Python and ML frameworks like TensorFlow, PyTorch, and OpenAI APIs, and possesses agile and proactive thinking.
